In [10]:
%load_ext autoreload
%autoreload 2

In [12]:
from networkx import volume
import pandas as pd
import pickle
close_df = pd.read_pickle('/Z1_3500/data/eod/close.pk')
volume_df = pd.read_pickle('/Z1_3500/data/eod/volume.pk')

In [7]:
vwap_df = (close_df * volume_df).cumsum() / volume_df.cumsum()

In [ ]:

with open('vwap.pk', 'wb') as f:
    pickle.dump(vwap_df, f)

In [ ]:
# Compute the forward return
forward_return_df = close_df.pct_change().shift(-1)
with open('../data/return.pk', 'wb') as f:
    pickle.dump(forward_return_df, f)

/tmp/ipykernel_2883145/2581389097.py:1: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  return_df = close_df.pct_change().shift(-1)


In [14]:
# copy open.pk to working directory
import shutil
shutil.copyfile('/Z1_3500/data/eod/open.pk', 'open.pk')
shutil.copyfile('/Z1_3500/data/eod/close.pk', 'close.pk')
shutil.copyfile('/Z1_3500/data/eod/high.pk', 'high.pk')
shutil.copyfile('/Z1_3500/data/eod/low.pk', 'low.pk')
shutil.copyfile('/Z1_3500/data/eod/volume.pk', 'volume.pk')

'volume.pk'

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import logging
import os

seed = 42
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_iterations = 50
batch_size = 64
save_every = 1
eval_every = 1
logdir = '../logs'
ckpt_dir = '../checkpoints'
os.makedirs(ckpt_dir, exist_ok=True)

2025-06-04 09:46:21.481575: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749030381.494487  778357 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749030381.498461  778357 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749030381.510510  778357 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749030381.510521  778357 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749030381.510523  778357 computation_placer.cc:177] computation placer alr

In [2]:
logfile = os.path.join(logdir, "training.log")
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler(logfile),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)
writer = SummaryWriter(logdir)

In [3]:
from agent.alpha_pool import AlphaPool
from agent.data import DataContainer
from agent.environment import AlphaEnv
from agent.game import selfplay
from agent.network import Network
from agent.train import train_network, inference


initial_rpns = [
        ["close", "dt5", "mean"],
        # ["volume", "log"],
        ["vwap", "close", "sub"]
    ]
stock_data = DataContainer(device=device)
pool = AlphaPool(initial_rpns, stock_data)
env = AlphaEnv(pool, device, False)
net = Network(env.action_size, 128).to(device)

In [4]:
%load_ext line_profiler

In [5]:
from agent.alpha_pool import AlphaPool
from agent.game import selfplay
from agent.mcts import mcts_search
%lprun -u 1 -f mcts_search selfplay(seed, net, device, env, 50, 1000, 1.0, 5, 5, logger)

2025-05-30 04:36:40,881 [INFO] Invalid expression: [BEG, high, -30.0, Mul, high, Add, 0.01, Sub, -10.0, volume, Less, 0.01, Add, 40, Sum, low, high, vwap, 5, Mean]
2025-05-30 04:36:40,884 [INFO] Reward: -1.001
2025-05-30 04:36:42,732 [INFO] Invalid expression: [BEG, -30.0, open, 1, Max, 1, Median, -0.5, Pow, vwap, volume, close, open, Greater, -0.5, Sub, 20, Median, Greater, Add]
2025-05-30 04:36:42,734 [INFO] Reward: -1.001
2025-05-30 04:36:48,272 [INFO] Expression: Log(Greater(Greater(Abs(Greater(Div(Add($vwap, 5.0), -1.0), 2.0)), $volume), Add(10.0, Add($open, Greater(10.0, $open)))))
2025-05-30 04:36:48,274 [INFO] Reward: 0.02910488028559461
2025-05-30 04:36:50,155 [INFO] Invalid expression: [BEG, vwap, close, -10.0, Less, 1, Min, high, 30.0, Div, 5.0, Div, 30.0, Mul, low, volume, 0.01, open, 10.0, vwap]
2025-05-30 04:36:50,156 [INFO] Reward: -1.001
2025-05-30 04:36:52,105 [INFO] Invalid expression: [BEG, -2.0, volume, Less, Abs, 0.01, close, volume, 2.0, Greater, -5.0, Add, close,

Timer unit: 1 s

Total time: 13.3059 s
File: /home/jy_zhou/python_project/src/agent/mcts.py
Function: mcts_search at line 121

Line #      Hits         Time  Per Hit   % Time  Line Contents
   121                                           def mcts_search(
   122                                                   env: AlphaEnv,
   123                                                   eval_func,
   124                                                   root_node,
   125                                                   c_puct_base,
   126                                                   c_puct_init,
   127                                                   num_simulations,
   128                                                   root_noise,
   129                                                   warm_up,
   130                                                   deterministic,
   131                                           ):
   132                                               
   133   

In [13]:
from agent.expression import Div, Greater, Less, Log, Mean, Mul, Pow, RollingOperator, Sub, Var
from agent.tokens import ConstantToken, DeltaTimeToken, FeatureToken, FeatureType, OperatorToken
from agent.expr_tree import ExprTree
# Mul($vwap, Mul(Log($vwap), Mul(-1.0, $vwap)))
tree = ExprTree()
tree.push(FeatureToken(FeatureType.OPEN))
tree.push(FeatureToken(FeatureType.OPEN))
tree.push(OperatorToken(Mul))

tree.get_stack()

Mul($open, $open)

In [14]:
tree.get_stack().evaluate(stock_data)

tensor([[ 255.6801,  207.0721,  242.7364,  ...,       nan,       nan,
               nan],
        [ 251.2225,  213.1600,  244.9225,  ...,       nan,       nan,
               nan],
        [ 242.1136,  203.3476,  270.6025,  ...,       nan,       nan,
               nan],
        ...,
        [ 294.4656,  386.1225,  382.9849,  ..., 1814.7599, 2822.7971,
         4692.2500],
        [ 280.8976,  374.4225,  375.5844,  ..., 1804.5504, 2782.5625,
         4622.6396],
        [ 284.2596,  366.3396,  382.2025,  ..., 1806.2500, 2798.4102,
         4761.0000]], device='cuda:0')

In [15]:
pool.compute_marginal_contribution(tree.get_stack())

0.005093226259271706

In [29]:
print(env._actions)

[Abs, Log, Add, Sub, Mul, Div, Greater, Less, Pow, Mean, Median, Sum, Std, Var, Max, Min, open, high, low, close, vwap, volume, 1, 5, 10, 20, 40, -30.0, -10.0, -5.0, -2.0, -1.0, -0.5, -0.01, 0.01, 0.5, 1.0, 2.0, 5.0, 10.0, 30.0, SEP]


In [9]:
env.reset()
env.step(16)
env.step(27)
env.step(8)
env.step(41)

({'state': array([16, 27,  8, 41,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0], dtype=uint8),
  'stack_len': 1},
 0.009638959290070425,
 True,
 False,
 {'action_masks': array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        dtype=uint8)})

In [10]:
step_rewards = [-0.001, -0.001, -0.001, 0.00963]

In [11]:
returns = []
R = 0.0
for r in reversed(step_rewards):
    R = r + 0.99 * R
    returns.append(R)
returns = list(reversed(returns))

In [12]:
returns

[0.006373879369999998, 0.007448362999999999, 0.008533699999999998, 0.00963]

In [8]:
# save the buffer
import pickle

In [9]:

with open('buffer.pkl', 'wb') as f:
    pickle.dump(buffer, f)

In [5]:
# load the buffer
with open('buffer.pkl', 'rb') as f:
    buffer = pickle.load(f)

In [10]:
inference(env, net, device, 5, 100, 1.0, 5, True)

RPN: $open
Reward: 0.009732796508229941
RPN: $low
Reward: 0.01375559229602325
RPN: $open
Reward: 0.009732796508229941
RPN: $close
Reward: 0.011164936323663877
RPN: [SEP, 1.0, volume, -0.01, vwap, -10.0, open, 40, Sum, open, 40, Sum, open, 40, Min]
Reward: -5.0
Mean Reward: -0.9911


-0.9911227756727706

In [11]:
optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)
train_network(
    net,
    buffer,
    optimizer,
    batch_size,
    device=device,
    num_epochs=30,
    writer=writer,
    logger=logger,
    )

2025-05-07 09:57:41,034 [INFO] [Train] Epoch 1/30 | Avg Loss: 16.9952
2025-05-07 09:57:41,361 [INFO] [Train] Epoch 2/30 | Avg Loss: 9.3666
2025-05-07 09:57:41,686 [INFO] [Train] Epoch 3/30 | Avg Loss: 5.7295
2025-05-07 09:57:42,018 [INFO] [Train] Epoch 4/30 | Avg Loss: 3.7801
2025-05-07 09:57:42,340 [INFO] [Train] Epoch 5/30 | Avg Loss: 3.2504
2025-05-07 09:57:42,664 [INFO] [Train] Epoch 6/30 | Avg Loss: 3.1857
2025-05-07 09:57:42,989 [INFO] [Train] Epoch 7/30 | Avg Loss: 3.0668
2025-05-07 09:57:43,318 [INFO] [Train] Epoch 8/30 | Avg Loss: 2.9407
2025-05-07 09:57:43,644 [INFO] [Train] Epoch 9/30 | Avg Loss: 2.8775
2025-05-07 09:57:43,965 [INFO] [Train] Epoch 10/30 | Avg Loss: 2.8394
2025-05-07 09:57:44,296 [INFO] [Train] Epoch 11/30 | Avg Loss: 2.7630
2025-05-07 09:57:44,625 [INFO] [Train] Epoch 12/30 | Avg Loss: 2.7679
2025-05-07 09:57:44,948 [INFO] [Train] Epoch 13/30 | Avg Loss: 2.7468
2025-05-07 09:57:45,276 [INFO] [Train] Epoch 14/30 | Avg Loss: 2.7200
2025-05-07 09:57:45,600 [INF

270

In [12]:
inference(env, net, device, 5, 100, 1.0, 5, True)

RPN: [SEP, volume, -10.0, volume, open, -0.01, low, open, open, open, low, open, open, open, open]
Reward: -5.0
RPN: [SEP, 0.01, vwap, open, high, volume, open, open, open, open, open, open, open, open, open]
Reward: -5.0
RPN: [SEP, -10.0, open, volume, 0.5, open, low, open, open, open, open, open, low, open, open]
Reward: -5.0
RPN: [SEP, low, -2.0, vwap, -30.0, low, open, open, open, open, open, open, low, open, open]
Reward: -5.0
RPN: [SEP, close, vwap, 0.5, vwap, -5.0, open, open, open, open, open, low, open, open, open]
Reward: -5.0
Mean Reward: -5.0000


-5.0